In [ ]:
### Non fine-tuned small open-source LLMs (based on Unsloth compatibility)

# 1. Llama 3.1 (8B)
# 2. Mistral Small (24B)
# 3. Gemma Instruct (2B)

In [1]:
from together import Together
import pandas as pd
from openai import OpenAI
import os
from dotenv import load_dotenv

In [ ]:
load_dotenv()

In [19]:
client = Together(api_key=os.getenv("API_KEY"))

In [8]:
instructions = """
You are a helpful research assistant. Please classify the following texts which contain text excerpts from EU Commissioners speeches into one, and only one, of the following categories: 

Prescriptive: Statements where the EU sets guidelines and recommendations for addressing issues, promoting a top-down approach that urges member states to align with its directives, preferences and recommendations. These are statements which contain clear and concrete policy guidelines detailing what actions member states should undertake. 

Discretionary: Statements where the EU encourages member states to adopt their own strategies to tackle challenges, allowing them more flexibility in implementing solutions that suit their needs, without necessarily providing concrete policy guidelines. These statements may also express support for bottom-up initiatives from member states, social actors, or private corporations, as well as for multilateral collaborations and engagements.

None: These are statements that do not fit in either category. 

Please limit your answer to these categories "Prescriptive", "Discretionary", "None" and do not motivate your answer. This is the text: 
"""


In [9]:
df = pd.read_csv("https://raw.githubusercontent.com/joshcova/Open_source_LLM_finetuning/refs/heads/main/data/eu_speeches_annotation_manual_2.csv")

In [21]:
messages = [ 
    {"role": "system", "content": instructions},
    {"role": "user", "content": df['text'].values[0]}
]

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
    messages=messages,
    seed=42,
    temperature=0.0,
)

results = response.choices[0].message.content
results

'Prescriptive'

In [22]:
def classify_text(text):
    messages = [ 
        {"role": "system", "content": instructions},
        {"role": "user", "content": text}
    ]

    response = client.chat.completions.create(
        model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
        messages=messages,
        seed=42,
        temperature=0.0,
    )

    results = response.choices[0].message.content
    return results

In [23]:
results = df.text.apply(classify_text)

In [24]:
df.loc[:, "results_llama_8B"] = results

In [ ]:
df["results_llama_8B"].value_counts()